In [56]:
import numpy as np
from tqdm.notebook import tnrange
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
import numpy as np
from scipy.special import erfc

def S2B(symbol, table):
    b = table[symbol]
    B = np.array(list(b)).astype(np.float32)
    return B

def probB(x, N0):
    d1 = abs(-1-x)
    d2 = abs(1-x)
    p0 = np.e**(-d1/N0)/(np.e**(-d1/N0)+np.e**(-d2/N0))
    p1 = 1-p0
    return [p0, p1]

def probS(S, N0, q, table, M):
    B = np.zeros((q, 2))
    for i in range(q):
        B[i] = probB(S[i], N0)
    P = np.zeros(M)
    for i in range(M):
        Bi = S2B(i, table)
        p = 1
        for j in range(q):
            p *=B[j, int(Bi[j])]
        P[i] = p
    m = np.max(P)
    P = np.log10(m/P)
    P[P>2]=2
    return P

def BPSK(message, M, table, SNRdB):
    Eb = 1
    N0 = Eb/10**(SNRdB/10)
    q = int(np.log2(M))
    Ns = len(message)
    Nb = Ns*q
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    noise = noise.reshape((Ns, q))
    X = np.zeros_like(noise)
    for i in range(Ns):
        X[i]= S2B(message[i], table)
    X[X==0]=-1
    Y = X+noise
    output = np.zeros((Ns, M))
    for i in range(Ns):
        output[i] = probS(Y[i], N0, q, table, M)
    return output

def HardDicision(Message):
    return np.argmin(Message, axis=1)

def score(X, Y):
    Z = np.zeros_like(X)
    Z[X!=Y]=1
    return np.sum(Z)

def predict(Q):
    message = np.argmin(Q, axis=0)
    X = 0
    for i in range(dc):
        X = gl.add(X, message[i])
    return message, X

def permutation(X, hi):
    Y = np.zeros_like(X)
    Y[0] = X[0]
    for i in range(1, X.shape[0]):
        Y[gl.multi(i, hi)] = X[i]
    return Y

def permutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], Y[i])
    return Z

def unpermutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], gl.reserve_e(Y[i]))
    return Z

def convert_vhdl(A, elem32=False):
    shape = A.shape
    s = "("
    if len(shape)==1:
        for i in A:
            si = int(i)
            s+=str(si)+", "
        s=s[:-2]+")"
        return s
    if len(shape)==2:
        for row in A:
            print(convert_vhdl(row)+",")

def debug(name, value, debug=False):
    if debug:
        print(name+"#"*30)
        print(value)
        
def row_selection(Q, H_i, H_non_zeros_index_i):
    Qi = np.zeros((q, dc))
    Qi = Q[:,H_non_zeros_index_i]
    Qi = permutation2d(Qi, H_i[H_non_zeros_index_i])
    return Qi

def row_update(Q, Qi2, H_i, H_non_zeros_index_i):
    Qi2 = unpermutation2d(Qi2,H_i[H_non_zeros_index_i])
    Q[:, H_non_zeros_index_i] = Qi2
    return Q
    
def CN(Q, A= 0.5,  dg=False):
    #print(convert_vhdl(Q))
    zn = np.argmin(Q, axis=0)
    debug("Zn", zn, dg)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
            
    debug("dQ", dQ, dg)
    B = 0
    for i in range(dc):
        B = gl.add(B, zn[i])
    debug("B", B, True)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], sortA[1], col

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    debug("m", m, True)
    debug("m0", m[:, 0], dg)
#     print(convert_vhdl(m[:, 0]))        
#     print("m1"+"#"*30)
#     print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*100000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 2]!=m[j-1,2]:
                conf[gl.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    debug("conf", conf, True)
    extra_columm = np.zeros((q, 6))
    extra_columm[:, 0] = np.min(conf, axis=1)
    extra_columm[1:, 1] = m[:, 0]
    extra_columm[1:, 2] = m[:, 1]
    extra_columm[1:, 5] = m[:, 2]
    argmin=  np.argmin(conf, axis=1)
    for i in range(1, q):
        if argmin[i]==0:
            extra_columm[i, 3] = extra_columm[i, 4] = m[i-1, 2]
        else:
            d1 = m[argmin[i]-1, 2]
            d2 = m[gl.add(argmin[i], i)-1, 2]
            extra_columm[i, 3]=d1
            extra_columm[i, 4] =d2
    debug("extra_columm", extra_columm, True)
    dR = np.zeros_like(dQ)
    
    def dRij(extra_colummi, j):
        dQi, m1, m2, d1, d2, mcol = extra_colummi
        if d1==j or d2==j:
            if d1 == d2:
                return m2
            else:
                if mcol ==j:
                    return m2
                else:
                    return m1
        else:
            return dQi
        
    for i in range(q):
        for j in range(dc):
            dR[i, j] = dRij(extra_columm[i], j)
    debug("dR", dR, True)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl.add(i, B)
            index = gl.add(index, zn[j])
            R[index, j]= (A*dR[i,j]).astype(dtype)
            
    debug("R", R,dg) 
    return R

def quick_check(H, C):
    C = C.reshape((C.shape[0], 1))
    return gl.multi(np.array(H), C)

gl = GF([4,  1, 0])
print(gl.table)

['0000' '0001' '0010' '0100' '1000' '0011' '0110' '1100' '1011' '0101'
 '1010' '0111' '1110' '1111' '1101' '1001']


In [57]:
Scale = 50
dc = 5
q = 16
dtype = np.int16
#dtype = np.float32
H = np.load("H.npy").astype(np.int16)
m = H.shape[0]
n = H.shape[1]
k = n- m
print(m, n, k)
H_non_zeros_index  = np.zeros((m, dc), np.uint16)
for i in range(m):
    stt = 0
    for j in range(n):
        if H[i, j]!=0:
            H_non_zeros_index[i, stt] = j
            stt+=1           
for i in range(m):
    Hi = H_non_zeros_index[i]
    Hi2 = np.zeros_like(Hi)
    for j in range(dc):
        Hi2[j] = Hi[j]- j*(q-1)
    print(convert_vhdl(Hi2))
print("#"*20)
for i in range(m):
    print(convert_vhdl(H_non_zeros_index[i]))
print("#"*20)
for i in range(m):
    print(convert_vhdl(H[i][H_non_zeros_index[i]]))

30 75 45
(5, 12, 11, 6, 3)
(6, 13, 12, 7, 4)
(7, 14, 13, 8, 5)
(8, 0, 14, 9, 6)
(9, 1, 0, 10, 7)
(10, 2, 1, 11, 8)
(11, 3, 2, 12, 9)
(12, 4, 3, 13, 10)
(13, 5, 4, 14, 11)
(14, 6, 5, 0, 12)
(0, 7, 6, 1, 13)
(1, 8, 7, 2, 14)
(2, 9, 8, 3, 0)
(3, 10, 9, 4, 1)
(4, 11, 10, 5, 2)
(8, 6, 13, 12, 7)
(9, 7, 14, 13, 8)
(10, 8, 0, 14, 9)
(11, 9, 1, 0, 10)
(12, 10, 2, 1, 11)
(13, 11, 3, 2, 12)
(14, 12, 4, 3, 13)
(0, 13, 5, 4, 14)
(1, 14, 6, 5, 0)
(2, 0, 7, 6, 1)
(3, 1, 8, 7, 2)
(4, 2, 9, 8, 3)
(5, 3, 10, 9, 4)
(6, 4, 11, 10, 5)
(7, 5, 12, 11, 6)
####################
(5, 27, 41, 51, 63)
(6, 28, 42, 52, 64)
(7, 29, 43, 53, 65)
(8, 15, 44, 54, 66)
(9, 16, 30, 55, 67)
(10, 17, 31, 56, 68)
(11, 18, 32, 57, 69)
(12, 19, 33, 58, 70)
(13, 20, 34, 59, 71)
(14, 21, 35, 45, 72)
(0, 22, 36, 46, 73)
(1, 23, 37, 47, 74)
(2, 24, 38, 48, 60)
(3, 25, 39, 49, 61)
(4, 26, 40, 50, 62)
(8, 21, 43, 57, 67)
(9, 22, 44, 58, 68)
(10, 23, 30, 59, 69)
(11, 24, 31, 45, 70)
(12, 25, 32, 46, 71)
(13, 26, 33, 47, 72)
(14, 27, 34

In [14]:
import time
def decoder2(Q, A):
    Rmna = np.zeros((m, q, dc), dtype)
    citer = 0
    for t in range(5):
        citer+=1
        stt = 0
        for i in range(m):
#             print(i)
#             print( H_non_zeros_index[i])
#             print(H[i][H_non_zeros_index[i]])
            Qmnai = permutation2d(Q[:, H_non_zeros_index[i]], H[i][H_non_zeros_index[i]])-Rmna[i]
#             convert_vhdl( Q[:, H_non_zeros_index[i]].transpose())
#             print("#"*20)
            minQ = np.min(Qmnai, axis=0)
            Qmnai = Qmnai - minQ
#             convert_vhdl( Qmnai.transpose())
#             print("#"*20)
            Rmna[i] = CN(Qmnai, A)
#             convert_vhdl( Rmna[i].transpose())
            QRmni = Rmna[i]+ Qmnai
            Q[:, H_non_zeros_index[i]] = unpermutation2d(QRmni,  H[i][H_non_zeros_index[i]])
        y, s = predict(Q)
        s = np.sum(y)
        if s == 0:
            break
    return y, citer


In [15]:
def test(SNRdB):
    samples=int(1e4)
    Ne = 0
    F_e = 0
    S_e = 0
    NF = 0
    Nb = 0
    for i in tnrange(samples):
        NF+=1
        Nb+=n
        xi = np.zeros(n, np.int16)
        Qi = BPSK(xi, q, gl.table, SNRdB)
        Qi = Qi.transpose()
        Qi = (Qi*Scale).astype(np.int16)
        y, inters = decoder2(Qi, 1)
        e = score(xi, y)
        if e!=0:
            F_e+=1
        S_e+=e
        if F_e>100:
            break
    return NF, F_e,Nb, S_e, np.log10(F_e/ NF), np.log10(S_e/ Nb)

In [16]:
test(1)

(3918, 101, 293850, 555, -1.588743057876075, -2.7238327119277415)

In [36]:
print(results)

[[ 54 365]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]
 [  0   0]]


In [58]:
xi = np.zeros(n, np.int16)
Qi = BPSK(xi, q, gl.table, 1)
Qi = Qi.transpose()
Qi = (Qi*Scale).astype(np.int16)


In [59]:
print(Qi[0])
np.save("Qi.npy", Qi)
y, inters = decoder2(Qi.copy(), 1)
print(y)

[ 0  0  0  0  0  0  0 22  0 25  0  0  8  0  0  0  5  0  0  0  8  0  0  0
  0  9  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0 27  0  0  0
  4  0 24  0  1  1  0 30  0  0  6  3  0  0  0 19  0  6  0  0  0  0  0  0
  0  0  0]
B##############################
5
m##############################
[[ 54. 100.   1.]
 [ 81. 100.   2.]
 [ 35.  99.   2.]
 [ 54. 100.   4.]
 [ 19.  66.   4.]
 [ 54.  54.   0.]
 [ 42.  44.   3.]
 [ 54.  54.   0.]
 [ 11.  38.   0.]
 [ 54.  90.   3.]
 [ 63.  96.   2.]
 [ 54.  66.   2.]
 [ 54.  54.   1.]
 [ 26.  48.   2.]
 [  9.  54.   2.]]
conf##############################
[[0.0e+00 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05
  1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05]
 [5.4e+01 1.0e+05 8.1e+01 3.5e+01 5.4e+01 1.0e+05 6.3e+01 4.2e+01 5.4e+01
  1.0e+05 1.0e+05 1.0e+05 5.4e+01 1.0e+05 1.0e+05 1.0e+05]
 [8.1e+01 5.4e+01 1.0e+05 5.4e+01 5.4e+01 1.0e+05 1.0e+05 5.4e+01 5.4e+01
  6.3e+01 1.0e+05 1.0e+05 1.0e+05 5.4e+01 1.0e+05 1.0e+05]

m##############################
[[ 77.  88.   0.]
 [ 77.  79.   0.]
 [ 59.  98.   2.]
 [ 19.  93.   0.]
 [105. 120.   2.]
 [ 89. 102.   3.]
 [110. 123.   0.]
 [ 50. 113.   3.]
 [ 93. 103.   1.]
 [ 71.  74.   2.]
 [ 64.  69.   4.]
 [ 70.  92.   4.]
 [ 37.  59.   0.]
 [ 41.  74.   3.]
 [ 43.  99.   3.]]
conf##############################
[[0.00e+00 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05
  1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [7.70e+01 1.00e+05 1.05e+02 9.30e+01 4.30e+01 1.00e+05 8.90e+01 1.10e+02
  7.10e+01 1.00e+05 1.00e+05 1.00e+05 7.00e+01 1.00e+05 1.00e+05 1.00e+05]
 [7.70e+01 1.05e+02 1.00e+05 8.90e+01 7.10e+01 1.00e+05 1.00e+05 1.10e+02
  1.00e+05 9.30e+01 1.00e+05 1.00e+05 1.00e+05 4.10e+01 1.00e+05 1.00e+05]
 [5.90e+01 9.30e+01 8.90e+01 1.00e+05 1.00e+05 1.05e+02 1.00e+05 1.00e+05
  5.00e+01 1.00e+05 7.10e+01 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [1.90e+01 7.70e+01 7.70e+01 1.10e+02 1.00e+05 1.05e+02 8.90e+01 1

B##############################
3
m##############################
[[118. 135.   4.]
 [ 97. 111.   4.]
 [  2.  96.   0.]
 [ 90. 119.   0.]
 [118. 131.   3.]
 [ 49. 117.   0.]
 [ 11. 150.   0.]
 [109. 115.   0.]
 [ 77.  95.   2.]
 [ 86. 118.   2.]
 [ 71.  81.   0.]
 [ 78. 118.   0.]
 [ 56. 128.   0.]
 [104. 113.   0.]
 [104. 115.   0.]]
conf##############################
[[0.00e+00 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05
  1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [1.18e+02 1.00e+05 1.18e+02 7.70e+01 1.00e+05 1.00e+05 1.00e+05 1.00e+05
  1.09e+02 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [9.70e+01 1.18e+02 1.00e+05 1.00e+05 9.00e+01 1.00e+05 1.00e+05 1.00e+05
  1.00e+05 7.70e+01 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [2.00e+00 1.18e+02 9.70e+01 1.00e+05 1.00e+05 1.18e+02 1.00e+05 1.00e+05
  1.00e+05 1.00e+05 8.60e+01 1.00e+05 1.00e+05 1.00e+05 1.00e+05 1.00e+05]
 [9.00e+01 1.18e+02 9.70e+01 1.0

In [55]:
sub_Q = Qi[:, H_non_zeros_index[0]]
Qmnai = permutation2d(sub_Q, H[0][H_non_zeros_index[0]])
print(H[0][H_non_zeros_index[0]])
R = CN(Qmnai, 1)
print(R)
print("#"*20)

[ 6 13 12  7  4]
B##############################
7
m##############################
[[ 15.  35.   1.]
 [ 76. 100.   2.]
 [ 56. 100.   4.]
 [ 54.  70.   4.]
 [ 28.  68.   4.]
 [ 18.  45.   0.]
 [  2.  17.   4.]
 [ 43.  54.   3.]
 [ 54.  54.   0.]
 [ 35.  36.   0.]
 [ 68.  72.   2.]
 [ 13.  70.   2.]
 [ 21.  31.   2.]
 [ 54.  54.   1.]
 [ 54.  54.   1.]]
conf##############################
[[0.0e+00 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05
  1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05]
 [1.5e+01 1.0e+05 7.6e+01 5.6e+01 5.4e+01 1.0e+05 6.8e+01 5.4e+01 4.3e+01
  1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05]
 [7.6e+01 2.8e+01 1.0e+05 5.6e+01 5.4e+01 1.0e+05 1.0e+05 1.3e+01 5.4e+01
  6.8e+01 1.0e+05 1.0e+05 1.0e+05 5.4e+01 1.0e+05 1.0e+05]
 [5.6e+01 5.4e+01 7.6e+01 1.0e+05 1.0e+05 6.8e+01 1.0e+05 1.0e+05 4.3e+01
  1.0e+05 3.5e+01 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05]
 [5.4e+01 1.0e+05 7.6e+01 1.0e+05 1.0e+05 4.3e+01 1.8e+01 1.0e+05 1.0e+05
  5.4

In [38]:
for i in range(1, q):
    print(gl.multi(i, 6))

6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
